In [4]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd

import discretize 
from simpeg import maps
from simpeg.electromagnetics import time_domain as tdem

In [6]:
def read_data_file(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()

    # Remove empty lines and strip whitespace
    lines = [line.strip() for line in lines if line.strip()]
    
    # Separate the normalized and de-normalized blocks
    split_idx = next(i for i, line in enumerate(lines) if 'De-normalized Data' in line)

    raw_lines = lines[1:split_idx]  # skip header line
    denorm_lines = lines[split_idx+1:]

    def parse_block(block_lines):
        data = []
        for line in block_lines:
            if not line.startswith('%') and line.strip():
                values = [float(val) for val in line.split()]
                data.append(values)
        return np.array(data)

    raw_data = parse_block(raw_lines)
    denorm_data = parse_block(denorm_lines)

    # Optionally convert to pandas DataFrames
    num_fixed_cols = 5
    num_variable_cols = raw_data.shape[1] - num_fixed_cols
    variable_names = [f"TC1_{i+1}" for i in range(num_variable_cols)]
    col_names = ['X', 'Y', 'Z', 'Radar', 'TxRx_elevation'] + variable_names

    raw_df = pd.DataFrame(raw_data, columns=col_names)
    denorm_df = pd.DataFrame(denorm_data, columns=col_names)

    return raw_df, denorm_df

In [8]:
# Usage
raw_df, denorm_df = read_data_file("../dyang/Milligan_VTEM/INV_3D/Extracted_soundings.txt")


In [9]:
fig, ax = plt.subplots(1,1)

ax.plot(

,X,Y,Z,Radar,TxRx_elevation,TC1_1,TC1_2,TC1_3,TC1_4,TC1_5,...,TC1_18,TC1_19,TC1_20,TC1_21,TC1_22,TC1_23,TC1_24,TC1_25,TC1_26,TC1_27
0,433801.3,6109192.7,1146.2,82.5,1186.7,3.773245,2.599131,1.788588,1.207039,0.765195,...,0.001085,0.000083,-0.000459,-0.000822,-0.000887,-0.001016,-0.000544,-0.000442,-0.000058,-0.000335
1,434000.8,6109195.4,1148.3,74.1,1180.4,2.951651,1.899748,1.230008,0.790973,0.475560,...,0.001413,0.000232,-0.000232,-0.000379,-0.000378,-0.000592,-0.000531,-0.000335,-0.000299,-0.000141
2,434199.4,6109196.7,1135.9,85.1,1179.0,1.454023,1.008597,0.712932,0.509134,0.356247,...,0.003896,0.002118,0.001165,0.000531,0.000114,0.000109,0.000083,0.000009,-0.000122,-0.000014
3,434401.0,6109197.7,1128.5,85.3,1171.8,1.318170,0.948929,0.686561,0.496183,0.348632,...,0.004331,0.002600,0.001608,0.000930,0.000503,0.000212,0.000227,0.000072,0.000024,-0.000219
4,434601.9,6109201.7,1116.9,83.5,1158.4,1.828290,1.224779,0.827447,0.557233,0.360203,...,0.004220,0.002643,0.001515,0.001000,0.000600,0.000227,0.000258,0.000063,0.000069,-0.000019
5,434801.3,6109204.5,1112.3,80.7,1151.0,2.140841,1.460381,1.005699,0.690001,0.455089,...,0.006288,0.004286,0.002970,0.001930,0.000914,0.000565,0.000322,0.000204,0.000051,0.000185
6,435002.0,6109206.0,1066.7,104.2,1128.9,2.351066,1.748812,1.291883,0.940070,0.660427,...,0.008740,0.005958,0.003971,0.002569,0.001577,0.000938,0.000627,0.000354,0.000215,-0.000066
7,433801.2,6109397.8,1171.8,118.0,1247.8,1.648032,1.247139,0.930045,0.678711,0.472068,...,0.003222,0.001785,0.000866,0.000349,-0.000049,-0.000274,-0.000149,-0.000138,-0.000159,-0.000067
8,434001.0,6109395.9,1155.8,100.4,1214.2,1.822664,1.347866,0.981552,0.702572,0.481227,...,0.003739,0.002158,0.001248,0.000519,0.000155,-0.000010,0.000154,0.000163,0.000054,-0.000008
9,434200.0,6109402.1,1144.8,90.1,1192.9,1.441512,1.087349,0.816123,0.606649,0.439012,...,0.004082,0.002488,0.001410,0.000812,0.000370,0.000033,-0.000058,-0.000202,-0.000101,-0.000036
